In [ ]:
import os
import zipfile
import shutil
import cv2
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from skimage import io
from skimage.color import rgb2gray
import random
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from google.colab.patches import cv2_imshow
from tensorflow.keras import  layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from keras.applications import resnet
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras.applications.resnet import preprocess_input
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout, BatchNormalization

In [ ]:
BATCH_SIZE = 32
EPOCHS = 50
SEED = 49
H = 300
W = 400
IMAGE_SHAPE = (360,360,3)
N_CLASSES = 4
STEPS = 50

In [ ]:
train_data_gen = ImageDataGenerator(rescale=1/255)
valid_data_gen = ImageDataGenerator(rescale=1/255)

train_gen = train_data_gen.flow_from_directory(
    '/content/drive/MyDrive/Datasets/Chest_CTscan/train',
    target_size = (360,360),
    batch_size = BATCH_SIZE,
    class_mode = 'categorical'
)

valid_gen = valid_data_gen.flow_from_directory(
    '/content/drive/MyDrive/Datasets/Chest_CTscan/test',
    target_size = (360,360),
    batch_size = BATCH_SIZE,
    class_mode = 'categorical'
)

Found 670 images belonging to 4 classes.
Found 285 images belonging to 4 classes.


In [ ]:
res_model = ResNet50(include_top=False, 
                         pooling='avg', 
                         weights='imagenet', 
                         input_shape = (IMAGE_SHAPE))

for layer in res_model.layers:
    if 'conv5' not in layer.name:
        layer.trainable = False

resnet_model = Sequential()
resnet_model.add(res_model)
resnet_model.add(Dropout(0.4))
resnet_model.add(Flatten())
resnet_model.add(BatchNormalization())
resnet_model.add(Dropout(0.4))
resnet_model.add(Dense(N_CLASSES, activation='softmax'))
resnet_model.summary()

adam_opt = tf.keras.optimizers.Adam(learning_rate= 0.00001)

resnet_model.compile(optimizer = adam_opt, 
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 dropout_4 (Dropout)         (None, 2048)              0         
                                                                 
 flatten_2 (Flatten)         (None, 2048)              0         
                                                                 
 batch_normalization_2 (Batc  (None, 2048)             8192      
 hNormalization)                                                 
                                                                 
 dropout_5 (Dropout)         (None, 2048)              0         
                                                                 
 dense_2 (Dense)             (None, 4)                 8196      
                                                      

In [ ]:
checkpoint = ModelCheckpoint(filepath='./transfer_model.hdf5', 
                               monitor='val_loss', 
                               verbose = 1,
                               save_best_only=True)

#early_stopping = EarlyStopping(verbose=1, patience=6)

model_history = resnet_model.fit(train_gen,
                               epochs = EPOCHS,
                               verbose = 1,
                               validation_data = valid_gen,
                               callbacks = [checkpoint])

Epoch 1/50
21/21 [==============================] - ETA: 0s - loss: 0.3941 - accuracy: 0.8448
Epoch 1: val_loss improved from inf to 1.95976, saving model to ./transfer_model.hdf5
21/21 [==============================] - 14s 646ms/step - loss: 0.3941 - accuracy: 0.8448 - val_loss: 1.9598 - val_accuracy: 0.4722
Epoch 2/50
21/21 [==============================] - ETA: 0s - loss: 0.3924 - accuracy: 0.8537
Epoch 2: val_loss did not improve from 1.95976
21/21 [==============================] - 14s 655ms/step - loss: 0.3924 - accuracy: 0.8537 - val_loss: 2.1161 - val_accuracy: 0.4861
Epoch 3/50
21/21 [==============================] - ETA: 0s - loss: 0.3773 - accuracy: 0.8478
Epoch 3: val_loss improved from 1.95976 to 1.29800, saving model to ./transfer_model.hdf5
21/21 [==============================] - 14s 647ms/step - loss: 0.3773 - accuracy: 0.8478 - val_loss: 1.2980 - val_accuracy: 0.5417
Epoch 4/50
21/21 [==============================] - ETA: 0s - loss: 0.3100 - accuracy: 0.8776
Epoch

In [ ]:
resnet = ResNet50(input_shape=(224,224,3),include_top = False, weights = "imagenet")

for layers in resnet.layers:
        layers.trainable = False

In [ ]:
res_model = Flatten()(resnet.output)
res_model = Dense(1024, activation="relu")(res_model)
res_model = Dense(1, activation="sigmoid")(res_model)

model_resnet = Model(resnet.input, res_model)

model_resnet.compile(optimizer = RMSprop(learning_rate = 0.0001),
                   loss = "binary_crossentropy",
                   metrics=['acc'])

In [ ]:
resnet_fit = model_resnet.fit(train_gen, validation_data = valid_gen,
                              steps_per_epoch = 670,
                              validation_steps = 285,
                              verbose = 1, epochs = 50)

Epoch 1/50
133/670 [====>.........................] - ETA: 24s - loss: -61275.4062 - acc: 0.2030

670/670 [==============================] - 9s 13ms/step - loss: -61274.3711 - acc: 0.2030 - val_loss: -43145.8398 - val_acc: 0.1789


In [ ]:
BATCH_SIZE = 32
EPOCHS = 100
SEED = 49
H = 300 #Height
W = 400 #Width
IMAGE_SHAPE = (460,460,3)
N_CLASSES = 4
STEPS = 5

In [ ]:
model1.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_7 (Conv2D)           (None, 256, 256, 64)      4864      
                                                                 
 conv2d_8 (Conv2D)           (None, 256, 256, 128)     73856     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 128, 128, 128)    0         
 2D)                                                             
                                                                 
 conv2d_9 (Conv2D)           (None, 126, 126, 256)     295168    
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 63, 63, 256)      0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 61, 61, 128)      

In [ ]:

model1.compile(loss = 'binary_crossentropy',
             optimizer = 'Adam',
             metrics = ['accuracy'])

In [ ]:
early_stopping=tf.keras.callbacks.EarlyStopping(monitor="val_accuracy",patience=6)

In [ ]:
model1_fit = model1.fit(train_gen,
                     epochs = 40,
                     verbose = 1,
                     validation_data = valid_gen,
                      callbacks = early_stopping)


Epoch 1/40
20/20 [==============================] - 628s 31s/step - loss: -748929.2500 - accuracy: 0.1817 - val_loss: -7545811.0000 - val_accuracy: 0.1729
Epoch 2/40
20/20 [==============================] - 10s 506ms/step - loss: -2024244096.0000 - accuracy: 0.1817 - val_loss: -13798552576.0000 - val_accuracy: 0.1729
Epoch 3/40
20/20 [==============================] - 10s 509ms/step - loss: -504901337088.0000 - accuracy: 0.1817 - val_loss: -1773193920512.0000 - val_accuracy: 0.1729
Epoch 4/40
20/20 [==============================] - 10s 497ms/step - loss: -20752344547328.0000 - accuracy: 0.1817 - val_loss: -67534808154112.0000 - val_accuracy: 0.1729
Epoch 5/40
20/20 [==============================] - 11s 527ms/step - loss: -432238831337472.0000 - accuracy: 0.1817 - val_loss: -1135819133812736.0000 - val_accuracy: 0.1729
Epoch 6/40
20/20 [==============================] - 10s 514ms/step - loss: -5635907088547840.0000 - accuracy: 0.1817 - val_loss: -13093565357883392.0000 - val_accuracy:

In [ ]:
model1.save('caries_model1.h5')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
test_model = models.load_model('/content/caries_model1.h5')

In [ ]:
test_data_gen = ImageDataGenerator(rescale=1/255)

test_gen = test_data_gen.flow_from_directory(
    '/content/drive/MyDrive/Datasets/Chest_CTscan/test',
    target_size = (256,256),
    batch_size = 4,
    class_mode = 'binary'
)

Found 72 images belonging to 4 classes.


In [ ]:
score = test_model.evaluate(test_gen, verbose=1)

12/12 [==============================] - 1s 49ms/step - loss: 0.4742 - accuracy: 0.9167
